In [50]:
!pip install tensorflow transformers opencv-python pillow
!pip install --upgrade tensorflow
!pip install ipywidgets
!pip install notebook
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
C:\Users\shahj\anaconda3\Anaconda\Lib\site-packages\jupyterlab\debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
C:\Users\shahj\anaconda3\Anaconda\Lib\site-packages\jupyterlab\debuglog.py:55: UserWarning: ValueError: Please install nodejs >=20.0.0 before continuing. nodejs may be installed using conda or directly from the nodejs website.
  warnings.warn(msg[-1].strip())
C:\Users\shahj\anaconda3\Anaconda\Lib\site-packages\jupyterlab\debuglog.py:56: UserWarning: See the log file for details: C:\Users\shahj\AppData\Local\Temp\jupyterlab-debug-9_ts8sn4.log
  warnings.warn(f"See the log file for details: {log_path!s}")
usage: ju

In [1]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [3]:
import numpy as np
import cv2
import tensorflow as tf
from transformers import AutoImageProcessor, TFAutoModelForImageClassification
from PIL import Image

In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [19]:
image_processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression")
model = TFAutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [23]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
labels = model.config.id2label

emotion_scores = {
    'neutral': 0.5,
    'happy': 1.0,
    'surprise': 0.8,
    'anger': -0.5,
    'sadness': -0.7,
    'fear': -0.6,
    'disgust': -0.4
}

dynamic_score = 0
alpha = 0.1

while True:
    
    ret, frame = cap.read()
    if not ret:
        break
        
    #Face Detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    #If Multiple Faces
    for (x, y, w, h) in faces:
        
        #Draw a rectangle and crop the image
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face_image = frame[y:y + h, x:x + w]

        rgb_face = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_face)

        #Extract 
        inputs = image_processor(images=pil_image, return_tensors="tf")

        # Make Predictions
        predictions = model(inputs["pixel_values"])
        predicted_class = tf.argmax(predictions.logits, axis=-1).numpy()[0]
        
        emotion = labels[predicted_class]
        emotion_confidence = tf.nn.softmax(predictions.logits).numpy()[0][predicted_class]
        score_for_emotion = emotion_scores.get(emotion, 0) * emotion_confidence

        #Exponential Moving Average for the dynamic score
        dynamic_score = alpha * score_for_emotion + (1 - alpha) * dynamic_score

        cv2.putText(frame, f"Emotion: {emotion}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Confidence: {dynamic_score:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Emotion Detection and Dynamic Confidence Score', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

final_score = dynamic_score  
print(f"Final Confidence Score after the interview: {final_score:.2f}")

cap.release()
cv2.destroyAllWindows()

Final Confidence Score after the interview: 0.28
